In [7]:
## 统计A/B compartment 比例
import pandas as pd
df = pd.read_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed', sep="\t", names=['chr', 'start', 'end', 'pc1', 'tags'])
df_zero = df.query("pc1==0")
df_Acpt = df.query("pc1>0").copy()
df_Bcpt = df.query("pc1<0").copy()
df_Acpt['cpt'] = ['A']*df_Acpt.shape[0]
df_Acpt['number'] = [1]*df_Acpt.shape[0]
df_Bcpt['cpt'] = ['B']*df_Bcpt.shape[0]
df_Bcpt['number'] = [1]*df_Bcpt.shape[0]
df = pd.concat([df_Acpt, df_Bcpt], axis=0)
df.sort_values(by=['tags', 'chr', 'start'], inplace=True)
df_number = df.groupby(['tags', 'cpt'], as_index=False)['number'].sum()

sigs = ['7c', '7d', '8c', '8d']
times = ['ID', 'MD', 'SD', 'RW']
df_result = pd.DataFrame()
for s in sigs:
    for t in times:
        tgs = s+t
        df_tmp = df_number.query("tags==@tgs").copy()
        df_tmp.reset_index(inplace=True, drop=True)
        bin_sum_number = df_tmp['number'].sum()
        df_tmp['ratio'] = df_tmp['number']/bin_sum_number
        df_tmp['sig'] = [s]*df_tmp.shape[0]
        df_tmp['time'] = [t]*df_tmp.shape[0]
        df_tmp['center'] = df_tmp.loc[0, 'ratio']
        df_result = pd.concat([df_result, df_tmp], axis=0)
df_result.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt_number.bed", sep='\t', index=False)


In [30]:
# 统计A/B compartment 在两个亚基因组所占比例
df    = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed", sep="\t",  names=['chrs', 'start', 'end', 'pca1', 'tags'])
df    = df.query('pca1!=0').copy()
df_at = df.query("chrs.str.contains('A')", engine='python').copy()
df_dt = df.query("chrs.str.contains('D')", engine='python').copy()
df_at['subg'] = ['At']*df_at.shape[0]
df_dt['subg'] = ['Dt']*df_dt.shape[0]
df = pd.concat([df_at, df_dt], axis=0)
df['number'] = [1]*df.shape[0]
df_result = df.groupby(by=['tags', 'subg'], as_index=False)['number'].sum()
df_result['length'] = df_result['number']/10
df_result.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_cpt_length.bed", sep="\t", index=False)



In [ ]:
## 统计两个亚基因组A/B compartment 比例
import pandas as pd
df = pd.read_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed', sep="\t", names=['chr', 'start', 'end', 'pc1', 'tags'])
df['subg'] = df['chr'].apply(lambda x: 'At' if "Ghir_A" in x else 'Dt')
df_Acpt = df.query("pc1>0").copy()
df_Bcpt = df.query("pc1<0").copy()
df_Acpt['cpt'] = ['A']*df_Acpt.shape[0]
df_Acpt['number'] = [1]*df_Acpt.shape[0]
df_Bcpt['cpt'] = ['B']*df_Bcpt.shape[0] 
df_Bcpt['number'] = [1]*df_Bcpt.shape[0]
df = pd.concat([df_Acpt, df_Bcpt], axis=0)
df.sort_values(by=['tags', 'chr', 'start'], inplace=True)
df_number = df.groupby(['tags', 'subg', 'cpt'], as_index=False)['number'].sum()
sigs = ['7c', '7d', '8c', '8d']
times = ['ID', 'MD', 'SD', 'RW']
subgs = ['At', 'Dt']
df_result = pd.DataFrame()
for s in sigs:
    for t in times:
        for c in subgs:
            tgs = s+t
            df_tmp = df_number.query("tags==@tgs & subg==@c").copy()
            df_tmp.reset_index(inplace=True, drop=True)
            bin_sum_number = df_tmp['number'].sum()
            df_tmp['ratio'] = df_tmp['number']/bin_sum_number
            df_tmp['sig'] = [s]*df_tmp.shape[0]
            df_tmp['time'] = [t]*df_tmp.shape[0]
            df_tmp['center'] = df_tmp.loc[0, 'ratio']
            df_result = pd.concat([df_result, df_tmp], axis=0)
df_result.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt_at_dt_number.bed", sep='\t', index=False)


In [17]:
# 将多个compartment合并
samples = ['7cID', '7cMD', '7cSD', '7cRW', '7dID', '7dMD', '7dSD', '7dRW', '8cID', '8cMD', '8cSD', '8cRW', '8dID', '8dMD', '8dSD', '8dRW']
df = pd.DataFrame()
for s in samples:
    input_f = "G:/Billfish/boyang/HiC/compartment/100k/cworld/all_cpt/{}_cpt.bed".format(s)
    df_tmp = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', s])
    if df.shape[0] == 0:
        df = df_tmp.copy()
    else:
        df = pd.merge(df, df_tmp, how='outer', on=['chr', 'start', 'end'])
df_nonull = df.dropna()
df_null = pd.concat([df, df_nonull], axis=0)
df_null.drop_duplicates(keep=False, inplace=True)
print(df_null)

Empty DataFrame
Columns: [chr, start, end, 7cID, 7cMD, 7cSD, 7cRW, 7dID, 7dMD, 7dSD, 7dRW, 8cID, 8cMD, 8cSD, 8cRW, 8dID, 8dMD, 8dSD, 8dRW]
Index: []


In [48]:
## 分析每条染色体的A compartment 比例
import pandas as pd
import re
chromosome = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
length = [1178, 1081, 1131, 852, 1095, 1241, 978, 1224, 821, 1149, 1233, 1077, 1084, 633, 699, 527, 565, 630, 669, 593, 691, 529, 680, 730, 627, 634]
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed", sep="\t", names=['chrs', 'start', 'end', 'pca1', 'tags'])
df['number'] = [1]*df.shape[0]
df_A = df.query("pca1>0")
df_result = df_A.groupby(by=['chrs', 'tags'], as_index=False)['number'].sum()
pattern = re.compile(r'\w{2}')
for index, row in df_result.iterrows():
    sigs, times = pattern.findall(row['tags'])[0], pattern.findall(row['tags'])[1]
    df_result.loc[index, 'sig'] = sigs
    df_result.loc[index, 'time'] = times
df_result['length'] = df_result['chrs']
df_result['length'].replace(chromosome, length, inplace=True)
df_result['ratio'] = df_result['number']/df_result['length']
print(df_result)
df_result.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/all_A_cpt_lenght.bed", sep="\t", index=False)

         chrs  tags  number sig time  length     ratio
0    Ghir_A01  7cID     581  7c   ID    1178  0.493209
1    Ghir_A01  7cMD     574  7c   MD    1178  0.487267
2    Ghir_A01  7cRW     597  7c   RW    1178  0.506791
3    Ghir_A01  7cSD     619  7c   SD    1178  0.525467
4    Ghir_A01  7dID     621  7d   ID    1178  0.527165
..        ...   ...     ...  ..  ...     ...       ...
411  Ghir_D13  8cSD     325  8c   SD     634  0.512618
412  Ghir_D13  8dID     317  8d   ID     634  0.500000
413  Ghir_D13  8dMD     322  8d   MD     634  0.507886
414  Ghir_D13  8dRW     287  8d   RW     634  0.452681
415  Ghir_D13  8dSD     271  8d   SD     634  0.427445

[416 rows x 7 columns]


In [8]:
## 调整Short vs Long 格式
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/SVL/7d_7c_8d_8c_SVL.txt", sep="\t")
df_at = df.query("chrs.str.contains('A')", engine='python').copy()
df_dt = df.query("chrs.str.contains('D')", engine='python').copy()
df_at['subg'] = ['At']*df_at.shape[0]
df_dt['subg'] = ['Dt']*df_dt.shape[0]
df_at.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/SVL/7d_7c_8d_8c_At.txt", sep="\t", index=False)
df_dt.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/SVL/7d_7c_8d_8c_dt.txt", sep="\t", index=False)

df = pd.concat([df_at, df_dt], axis=0)
df.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/SVL/7d_7c_8d_8c_SVL.txt", sep="\t", index=False)

In [18]:
## 分析同时期，不同处理间A compartment差异
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt_atdt_number.bed", sep="\t")
df_7c_A = df.query("cpt=='A' & sig == '7c'").copy()
df_7d_A = df.query("cpt=='A' & sig == '7d'").copy()
df_8c_A = df.query("cpt=='A' & sig == '8c'").copy()
df_8d_A = df.query("cpt=='A' & sig == '8d'").copy()
df_7c_A.reset_index(inplace=True, drop=True)
df_7d_A.reset_index(inplace=True, drop=True)
df_8c_A.reset_index(inplace=True, drop=True)
df_8d_A.reset_index(inplace=True, drop=True)

df_7d_7c = df_7c_A[['sig', 'time', 'subg']].copy()
df_8d_8c = df_8c_A[['sig', 'time', 'subg']].copy()

df_7d_7c['A_cpt'] = df_7d_A['number'] - df_7c_A['number']
df_8d_8c['A_cpt'] = df_8d_A['number'] - df_8c_A['number']

df_result = pd.concat([df_7d_7c, df_8d_8c], axis=0)

df_result.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/A_compartment_change.bed", sep="\t", index=False)


In [10]:
## 统计分析在发育时期始终保持A 或者B, 以及由A 转向B 由B 转向A 再者A 转B 转A 或 B转A转B 的数目
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed", sep="\t", names=['chrs', 'start', 'end', 'pca1', 'tags'])
## 将数值替换为标签
df_A = df.query("pca1>0").copy()
df_B = df.query("pca1<0").copy()
df_A['pca1'] = ['A']*df_A.shape[0]
df_B['pca1'] = ['B']*df_B.shape[0]
df_AB = pd.concat([df_A, df_B], axis=0)

## 长数据转换为短数据
df_AB['chrom'] = df_AB['chrs'].astype(str) + "-" + df_AB['start'].astype(str) + "-" + df_AB['end'].astype(str)
df_wide = df_AB.pivot(index='chrom', columns='tags', values='pca1')
df_wide.reset_index(inplace=True)
## 去除空字符
df_wide.dropna(inplace=True)
df_wide['number'] = [1]*df_wide.shape[0]

# 分析四组样品动态变化的compartment数量
sample1 = ['chrom', '7cID', '7cMD', '7cSD', '7cRW', 'number']
sample2 = ['chrom', '7dID', '7dMD', '7dSD', '7dRW', 'number']
sample3 = ['chrom', '8cID', '8cMD', '8cSD', '8cRW', 'number']
sample4 = ['chrom', '8dID', '8dMD', '8dSD', '8dRW', 'number']

df_sample1,df_sample2,df_sample3,df_sample4 = df_wide[sample1].copy(),df_wide[sample2].copy(),df_wide[sample3].copy(),df_wide[sample4].copy()
df_sample1.columns = ['chrom', 'ID', 'MD', 'SD', 'RW', 'number']
df_sample2.columns = ['chrom', 'ID', 'MD', 'SD', 'RW', 'number']
df_sample3.columns = ['chrom', 'ID', 'MD', 'SD', 'RW', 'number']
df_sample4.columns = ['chrom', 'ID', 'MD', 'SD', 'RW', 'number']

df_sample1['tags'] = ['7c']*df_sample1.shape[0]
df_sample2['tags'] = ['7d']*df_sample2.shape[0]
df_sample3['tags'] = ['8c']*df_sample3.shape[0]
df_sample4['tags'] = ['8d']*df_sample4.shape[0]

df_sample = pd.concat([df_sample1, df_sample2, df_sample3, df_sample4], axis=0)
df_sample['sigs'] = df_sample['ID'] + df_sample['MD'] + df_sample['SD'] + df_sample['RW']
old_sigs = ['AAAA', 'AAAB', 'AABA', 'AABB', 'ABAA', 'ABAB', 'ABBA', 'ABBB', 'BAAA', 'BAAB', 'BABA', 'BABB', 'BBAA', 'BBAB', 'BBBA', 'BBBB']
new_sigs = ['AA', 'AB', 'ABA', 'AB', 'ABA', 'AB', 'ABA', 'AB', 'BA', 'BAB', 'BA', 'BAB', 'BA', 'BAB', 'BA', 'BB']
df_sample['sigs'].replace(old_sigs, new_sigs, inplace=True)
df_sample['subg'] = df_sample['chrom'].apply(lambda x : 'At' if 'A' in x else 'Dt')

## 分类 并统计
df_result = df_sample.groupby(by=['tags', 'sigs'], as_index=False)['number'].sum()
df_result.to_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/dynamic_cpt/all_sample_dynamic_cpt.bed', sep='\t', index=False)

## 分类，统计，亚组
df_result = df_sample.groupby(by=['tags', 'sigs', 'subg'], as_index=False)['number'].sum()
df_result.to_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/dynamic_cpt/all_sample_dynamic_cpt_subg.bed', sep='\t', index=False)

## 排序，并写入到文件
df_dynamic = df_sample.query("sigs!='AA' & sigs!='BB'")[['ID', 'MD', 'SD', 'RW', 'tags', 'sigs']].copy()
df_dynamic.sort_values(by=['tags'], inplace=True)
tags_list = ['7c', '7d', '8c', '8d']
sigs_list = ['AB', 'BA', 'ABA', 'BAB']
df_result = pd.DataFrame()
for t in tags_list:
    df_t = df_dynamic.query("tags==@t").copy()
    df_tmp = pd.DataFrame()
    for s in sigs_list:
        df_ts = df_t.query("sigs==@s").copy()
        df_tmp = pd.concat([df_tmp, df_ts], axis=0)
    df_tmp['x_lab'] = range(df_tmp.shape[0])
    df_result = pd.concat([df_result, df_tmp], axis=0)
df_long = pd.melt(df_result, id_vars=['tags','sigs', 'x_lab'])
df_long.to_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/dynamic_cpt/all_sample_dynamic_change.bed', sep="\t", index=False)

## 排序， 并写入文件At 
df_At = df_sample.query("subg=='At'").copy()
df_dynamic = df_At.query("sigs!='AA' & sigs!='BB'")[['ID', 'MD', 'SD', 'RW', 'tags', 'sigs']].copy()
df_dynamic.sort_values(by=['tags'], inplace=True)
tags_list = ['7c', '7d', '8c', '8d']
sigs_list = ['AB', 'BA', 'ABA', 'BAB']
df_result = pd.DataFrame()
for t in tags_list:
    df_t = df_dynamic.query("tags==@t").copy()
    df_tmp = pd.DataFrame()
    for s in sigs_list:
        df_ts = df_t.query("sigs==@s").copy()
        df_tmp = pd.concat([df_tmp, df_ts], axis=0)
    df_tmp['x_lab'] = range(df_tmp.shape[0])
    df_result = pd.concat([df_result, df_tmp], axis=0)
df_long = pd.melt(df_result, id_vars=['tags','sigs', 'x_lab'])
df_long.to_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/dynamic_cpt/all_sample_dynamic_change_At.bed', sep="\t", index=False)

## 排序， 并写入文件At 

df_Dt = df_sample.query("subg=='Dt'").copy()
df_dynamic = df_Dt.query("sigs!='AA' & sigs!='BB'")[['ID', 'MD', 'SD', 'RW', 'tags', 'sigs']].copy()
df_dynamic.sort_values(by=['tags'], inplace=True)
tags_list = ['7c', '7d', '8c', '8d']
sigs_list = ['AB', 'BA', 'ABA', 'BAB']
df_result = pd.DataFrame()
for t in tags_list:
    df_t = df_dynamic.query("tags==@t").copy()
    df_tmp = pd.DataFrame()
    for s in sigs_list:
        df_ts = df_t.query("sigs==@s").copy()
        df_tmp = pd.concat([df_tmp, df_ts], axis=0)
    df_tmp['x_lab'] = range(df_tmp.shape[0])
    df_result = pd.concat([df_result, df_tmp], axis=0)
df_long = pd.melt(df_result, id_vars=['tags','sigs', 'x_lab'])
df_long.to_csv('G:/Billfish/boyang/HiC/compartment/100k/cworld/dynamic_cpt/all_sample_dynamic_change_Dt.bed', sep="\t", index=False)

In [28]:
## 分析compartment 保守的程度
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed", sep="\t", names=['chrs', 'start', 'end', 'pca1', 'tags'])
## 将数值替换为标签
df_A = df.query("pca1>0").copy()
df_B = df.query("pca1<0").copy()
df_A['pca1'] = ['A']*df_A.shape[0]
df_B['pca1'] = ['B']*df_B.shape[0]
df_AB = pd.concat([df_A, df_B], axis=0)

## 长数据转换为短数据
df_AB['chrom'] = df_AB['chrs'].astype(str) + "-" + df_AB['start'].astype(str) + "-" + df_AB['end'].astype(str)
df_wide = df_AB.pivot(index='chrom', columns='tags', values='pca1')
df_wide.reset_index(inplace=True)
## 去除空字符
df_wide.dropna(inplace=True)
df_wide.columns = ['chrom', 'cID7', 'cMD7', 'cRW7', 'cSD7', 'dID7', 'dMD7', 'dRW7', 'dSD7', 'cID8', 'cMD8', 'cRW8', 'cSD8', 'dID8', 'dMD8', 'dRW8', 'dSD8']
df_wide['subg'] = df_wide['chrom'].apply(lambda x : 'At' if 'A' in x else 'Dt')

#at subgenome
df_7c_A_at = df_wide.query("cID7=='A' and cMD7=='A' and cSD7=='A' and cRW7=='A' and subg.str.contains('A')", engine='python')
df_7c_B_at = df_wide.query("cID7=='B' and cMD7=='B' and cSD7=='B' and cRW7=='B' and subg.str.contains('A')", engine='python')

df_7d_A_at = df_wide.query("dID7=='A' and dMD7=='A' and dSD7=='A' and dRW7=='A' and subg.str.contains('A')", engine='python')
df_7d_B_at = df_wide.query("dID7=='B' and dMD7=='B' and dSD7=='B' and dRW7=='B' and subg.str.contains('A')", engine='python')

df_7c_7d_A_at = df_wide.query("cID7=='A' and cMD7=='A' and cSD7=='A' and cRW7=='A' and dID7=='A' and dMD7=='A' and dSD7=='A' and dRW7=='A' and subg.str.contains('A')", engine='python')
df_7c_7d_B_at = df_wide.query("cID7=='B' and cMD7=='B' and cSD7=='B' and cRW7=='B' and dID7=='B' and dMD7=='B' and dSD7=='B' and dRW7=='B' and subg.str.contains('A')", engine='python')

df_8c_A_at = df_wide.query("cID8=='A' and cMD8=='A' and cSD8=='A' and cRW8=='A' and subg.str.contains('A')", engine='python')
df_8c_B_at = df_wide.query("cID8=='B' and cMD8=='B' and cSD8=='B' and cRW8=='B' and subg.str.contains('A')", engine='python')

df_8d_A_at = df_wide.query("dID8=='A' and dMD8=='A' and dSD8=='A' and dRW8=='A' and subg.str.contains('A')", engine='python')
df_8d_B_at = df_wide.query("dID8=='B' and dMD8=='B' and dSD8=='B' and dRW8=='B' and subg.str.contains('A')", engine='python')

df_8c_8d_A_at = df_wide.query("cID8=='A' and cMD8=='A' and cSD8=='A' and cRW8=='A' and dID8=='A' and dMD8=='A' and dSD8=='A' and dRW8=='A' and subg.str.contains('A')", engine='python')
df_8c_8d_B_at = df_wide.query("cID8=='B' and cMD8=='B' and cSD8=='B' and cRW8=='B' and dID8=='B' and dMD8=='B' and dSD8=='B' and dRW8=='B' and subg.str.contains('A')", engine='python')

df_7c_A = pd.concat([df_7c_A_at, df_7d_A_at, df_7c_7d_A_at], axis=0)
df_7c_B = pd.concat([df_7c_B_at, df_7d_B_at, df_7c_7d_B_at], axis=0)
df_7c_A.drop_duplicates(keep=False, inplace=True)
df_7c_B.drop_duplicates(keep=False, inplace=True)


df_8c_A = pd.concat([df_8c_A_at, df_8d_A_at, df_8c_8d_A_at], axis=0)
df_8c_B = pd.concat([df_8c_B_at, df_8d_B_at, df_8c_8d_B_at], axis=0)
df_8c_A.drop_duplicates(keep=False, inplace=True)
df_8c_B.drop_duplicates(keep=False, inplace=True)

print("7c_A:", df_7c_A_at.shape[0])
print("7c_B:", df_7c_B_at.shape[0])
print("7d_A:", df_7d_A_at.shape[0])
print("7d_B:", df_7d_B_at.shape[0])
print("7c_7d_A:", df_7c_7d_A_at.shape[0])
print("7c_7d_B:", df_7c_7d_B_at.shape[0])
print(df_7c_A.shape[0], df_7c_B.shape[0])


print("8c_A:", df_8c_A_at.shape[0])
print("8c_B:", df_8c_B_at.shape[0])
print("8d_A:", df_8d_A_at.shape[0])
print("8d_B:", df_8d_B_at.shape[0])
print("8c_8d_A:", df_8c_8d_A_at.shape[0])
print("8c_8d_B:", df_8c_8d_B_at.shape[0])
print(df_8c_A.shape[0], df_8c_B.shape[0])


# Dt subgenome
# df_7c_A_dt = df_wide.query("cID7=='A' and cMD7=='A' and cSD7=='A' and cRW7=='A' and subg.str.contains('D')", engine='python')
# df_7c_B_dt = df_wide.query("cID7=='B' and cMD7=='B' and cSD7=='B' and cRW7=='B' and subg.str.contains('D')", engine='python')

# df_7d_A_dt = df_wide.query("dID7=='A' and dMD7=='A' and dSD7=='A' and dRW7=='A' and subg.str.contains('D')", engine='python')
# df_7d_B_dt = df_wide.query("dID7=='B' and dMD7=='B' and dSD7=='B' and dRW7=='B' and subg.str.contains('D')", engine='python')

# df_7c_7d_A_dt = df_wide.query("cID7=='A' and cMD7=='A' and cSD7=='A' and cRW7=='A' and dID7=='A' and dMD7=='A' and dSD7=='A' and dRW7=='A' and subg.str.contains('D')", engine='python')
# df_7c_7d_B_dt = df_wide.query("cID7=='B' and cMD7=='B' and cSD7=='B' and cRW7=='B' and dID7=='B' and dMD7=='B' and dSD7=='B' and dRW7=='B' and subg.str.contains('D')", engine='python')

# df_8c_A_dt = df_wide.query("cID8=='A' and cMD8=='A' and cSD8=='A' and cRW8=='A' and subg.str.contains('D')", engine='python')
# df_8c_B_dt = df_wide.query("cID8=='B' and cMD8=='B' and cSD8=='B' and cRW8=='B' and subg.str.contains('D')", engine='python')

# df_8d_A_dt = df_wide.query("dID8=='A' and dMD8=='A' and dSD8=='A' and dRW8=='A' and subg.str.contains('D')", engine='python')
# df_8d_B_dt = df_wide.query("dID8=='B' and dMD8=='B' and dSD8=='B' and dRW8=='B' and subg.str.contains('D')", engine='python')

# df_8c_8d_A_dt = df_wide.query("cID8=='A' and cMD8=='A' and cSD8=='A' and cRW8=='A' and dID8=='A' and dMD8=='A' and dSD8=='A' and dRW8=='A' and subg.str.contains('D')", engine='python')
# df_8c_8d_B_dt = df_wide.query("cID8=='B' and cMD8=='B' and cSD8=='B' and cRW8=='B' and dID8=='B' and dMD8=='B' and dSD8=='B' and dRW8=='B' and subg.str.contains('D')", engine='python')

# df_7c_A = pd.concat([df_7c_A_dt, df_7d_A_dt, df_7c_7d_A_dt], axis=0)
# df_7c_B = pd.concat([df_7c_B_dt, df_7d_B_dt, df_7c_7d_B_dt], axis=0)
# df_7c_A.drop_duplicates(keep=False, inplace=True)
# df_7c_B.drop_duplicates(keep=False, inplace=True)


# df_8c_A = pd.concat([df_8c_A_dt, df_8d_A_dt, df_8c_8d_A_dt], axis=0)
# df_8c_B = pd.concat([df_8c_B_dt, df_8d_B_dt, df_8c_8d_B_dt], axis=0)
# df_8c_A.drop_duplicates(keep=False, inplace=True)
# df_8c_B.drop_duplicates(keep=False, inplace=True)

# print("7c_A:", df_7c_A_dt.shape[0])
# print("7c_B:", df_7c_B_dt.shape[0])
# print("7d_A:", df_7d_A_dt.shape[0])
# print("7d_B:", df_7d_B_dt.shape[0])
# print("7c_7d_A:", df_7c_7d_A_dt.shape[0])
# print("7c_7d_B:", df_7c_7d_B_dt.shape[0])
# print(df_7c_A.shape[0], df_7c_B.shape[0])


# print("8c_A:", df_8c_A_dt.shape[0])
# print("8c_B:", df_8c_B_dt.shape[0])
# print("8d_A:", df_8d_A_dt.shape[0])
# print("8d_B:", df_8d_B_dt.shape[0])
# print("8c_8d_A:", df_8c_8d_A_dt.shape[0])
# print("8c_8d_B:", df_8c_8d_B_dt.shape[0])
# print(df_8c_A.shape[0], df_8c_B.shape[0])






7c_A: 6586
7c_B: 6939
7d_A: 6075
7d_B: 6917
7c_7d_A: 6041
7c_7d_B: 6821
579 214
8c_A: 6596
8c_B: 7052
8d_A: 6351
8d_B: 7053
8c_8d_A: 6293
8c_8d_B: 6939
361 227


In [48]:
## 分析compartment 时期间变化所包含的基因
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt.bed", sep="\t", names=['chrs', 'start', 'end', 'pca1', 'tags'])
## 将数值替换为标签
df_A = df.query("pca1>0").copy()
df_B = df.query("pca1<0").copy()
df_A['pca1'] = ['A']*df_A.shape[0]
df_B['pca1'] = ['B']*df_B.shape[0]
df_AB = pd.concat([df_A, df_B], axis=0)

## 长数据转换为短数据
df_AB['chrom'] = df_AB['chrs'].astype(str) + "-" + df_AB['start'].astype(str) + "-" + df_AB['end'].astype(str)
df_wide = df_AB.pivot(index='chrom', columns='tags', values='pca1')
df_wide.reset_index(inplace=True)
## 去除空字符
df_wide.dropna(inplace=True)
df_wide[['chr', 'start', 'end']]= df_wide['chrom'].str.split('-', expand=True)
#df_wide.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/boyang_all_cpt_wide.bed", sep="\t", index=False, columns = ['chr', 'start', 'end', '7cID', '7cMD', '7cSD', '7cRW', '7dID', '7dMD', '7dSD', '7dRW', '8cID', '8cMD', '8cSD', '8cRW', '8dID', '8dMD', '8dSD', '8dRW'])

## 同一样品不同处理之间的差异
for i in ['7', '8']:
    for j in ['ID', 'MD', 'SD', 'RW']:
        sample1 = "{0}c{1}".format(i, j)
        sample2 = "{0}d{1}".format(i, j)
        output_f1 = "G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/{0}_{1}_AB.bed".format(sample1, sample2)
        output_f2 = "G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/{0}_{1}_BA.bed".format(sample1, sample2)
        #print(sample1, sample2)
        df_AB = df_wide[(df_wide[sample1]=='A') & (df_wide[sample2] == 'B')]
        df_BA = df_wide[(df_wide[sample1]=='B') & (df_wide[sample2] == 'A')]
        df_AB.to_csv(output_f1, sep="\t", index=False, header=False, columns = ['chr', 'start', 'end', sample1, sample2])
        df_BA.to_csv(output_f2, sep="\t", index=False, header=False, columns = ['chr', 'start', 'end', sample1, sample2])

## 同一处理不同样品之间的差异
for i in ['c', 'd']:
    for j in ['ID', 'MD', 'SD', 'RW']:
        sample1 = "7{0}{1}".format(i, j)
        sample2 = "8{0}{1}".format(i, j)
        output_f1 = "G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/{0}_{1}_AB.bed".format(sample1, sample2)
        output_f2 = "G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/{0}_{1}_BA.bed".format(sample1, sample2)
        #print(sample1, sample2)
        df_AB = df_wide[(df_wide[sample1]=='A') & (df_wide[sample2] == 'B')]
        df_BA = df_wide[(df_wide[sample1]=='B') & (df_wide[sample2] == 'A')]
        df_AB.to_csv(output_f1, sep="\t", index=False, header=False, columns = ['chr', 'start', 'end', sample1, sample2])
        df_BA.to_csv(output_f2, sep="\t", index=False, header=False, columns = ['chr', 'start', 'end', sample1, sample2])



In [60]:
# 将A/B compartment 转变区域所包含基因数目，整理为ggplot2 方便作图的格式
import pandas as pd
df = pd.read_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/all_switch_region_gene.bed", sep="\t", names=['number', 'tags'])
df[['sample1', 'sample2', 'cpt']] = df['tags'].str.split("_", expand = True)
for index, row in df.iterrows():
    one, two = row['sample1'], row['sample2']
    if ('7' in one and '7' in two) or ('8' in one and '8' in two):
        df.loc[index, 'sigs'] = "environments"
    else:
        df.loc[index, 'sigs'] = 'species'
    df.loc[index, 'tags'] = "{0}_{1}".format(one, two)
df.to_csv("G:/Billfish/boyang/HiC/compartment/100k/cworld/switch_compartment/all_switch_region_gene_format.bed", sep="\t", columns = ['tags', 'sigs', 'cpt', 'number'], index=False)

    

In [4]:
## 划分A/B compartment
import pandas as pd
samples = ['7cID', '7cMD', '7cRW', '7cSD', '7dID', '7dMD', '7dRW', '7dSD', '8cID', '8cMD', '8cRW', '8cSD', '8dID', '8dMD', '8dRW', '8dSD']
for s in samples:
    input_f = "G:/Billfish/boyang/HiC/compartment/100k/cworld/all_cpt/{}_cpt.bed".format(s)
    A_output_f = "G:/Billfish/boyang/HiC/compartment/100k/cworld/all_cpt/{}_Acpt.bed".format(s)
    B_output_f = "G:/Billfish/boyang/HiC/compartment/100k/cworld/all_cpt/{}_Bcpt.bed".format(s)
    df = pd.read_csv(input_f, sep="\t", names=['chr', 'start', 'end', 'pca1'])
    df_A = df.query("pca1>0").copy()
    df_B = df.query("pca1<0").copy()
    df_A.to_csv(A_output_f, sep="\t", index=False, header=False)
    df_B.to_csv(B_output_f, sep="\t", index=False, header=False)
    